### Ingestion del archivo "movie_cast.json"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
movies_casts_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("personId", IntegerType(), True),
    StructField("characterName", StringType(), True),
    StructField("genderId", IntegerType(), True),
    StructField("castOrder", IntegerType(), True)
])

In [0]:
# Se agrega la opcion("multiLine",True), ya que, el archivo es multilinea
movies_casts_df = spark.read \
                  .schema(movies_casts_schema) \
                  .option("multiLine", True) \
                  .json(f"{bronze_folder_path}/{v_file_date}/movie_cast.json") 

### Paso 2 - Renombrar las columnas y añadir columnas nuevas
1. "movieId" renombrar a "movie_id"
2. "personId" renombrar a "person_id"
3. "characterName" renombrar a "character_name"
4. Agregar las columnas de "ingestion_date" y "envoronment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
movies_casts_with_columns_df = add_ingestion_date(movies_casts_df) \
                                .withColumn("environment", lit(v_environment)) \
                                .withColumn("file_date", lit(v_file_date)) \
                                .withColumnsRenamed({"movieId":"movie_id", 
                                                     "personId":"person_id", 
                                                     "characterName":"character_name"})
                                     

### Paso 3 - Eliminar las columnas no deseadas del DataFrame

In [0]:
from pyspark.sql.functions import col

In [0]:
movies_casts_final_df = movies_casts_with_columns_df.drop(col("genderId"), col("castOrder"))

#### Paso 4 - Escribir la salida en formato "Parquet"

In [0]:
#overwrite_partition(movies_casts_final_df,"movie_silver","movies_casts","file_date")

In [0]:
# movies_casts_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/movies_casts")

# Crear una tabla en base al data frame movies_casts_final_df
# Creara la tabla movies_casts en a DB que se llame movie_silver
# Modulo: Spark SQL - Database / Table / View: seccion: Crear Tabla - Capa Silver

#movies_casts_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.movies_casts")

# Append, agrega los registros a la tabla existente, carga incremental de datos
#movies_casts_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movies_casts")

#Modulo: Delta Lake
merge_condition = "tgt.movie_id = src.movie_id AND tgt.person_id = src.person_id AND tgt.file_date = src.file_date" # Buscar por particion
merge_delta_lake(movies_casts_final_df,"movie_silver","movies_casts",silver_folder_path,merge_condition,"file_date")

In [0]:
%sql
-- Consultar la DB movie_silver, la tabla movies_casts
SELECT file_date, COUNT(1)
FROM movie_silver.movies_casts
GROUP BY 1;

In [0]:
display(spark.read.format("delta").load(f"{silver_folder_path}/movies_casts"))

In [0]:
dbutils.notebook.exit("Success")